---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

## **PandasDataFrameOutputParser**

* `Pandas DataFrame` = `Python` 프로그래밍 언어에서 널리 사용되는 `데이터 구조` = `데이터 조작 및 분석`을 위해 `흔히 사용` 

* `구조화된 데이터`를 다루기 위한 `포괄적인 도구 세트 제공` → **데이터 정제, 변환 및 분석과 같은 작업에 다양하게 활용 가능**

* 사용자가 임의의 Pandas DataFrame을 지정하고 해당 DataFrame에서 데이터를 추출하여 형식화된 사전 형태로 데이터를 조회할 수 있는 LLM을 요청할 수 있게 해줌

In [ ]:
# 1_새 프롬프트 생성하기
from langsmith import Client
from langchain.prompts import PromptTemplate                            # Langchain에서 프롬프트 템플릿을 만들기 위한 모듈 임포트
from langchain.prompts import ChatPromptTemplate
from langsmith import Client

import os
import json


# 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

In [3]:
import pprint                                                                 # 예쁘게 출력하기 위한 pprint 모듈 임포트
from typing import Any, Dict                                                  # 타입 힌트를 위해 Any와 Dict 타입 임포트

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser              # pandas DataFrame을 출력 형식으로 사용하기 위한 파서

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langsmith import traceable                                     # LangSmith 추적 설정

# LLM 초기화
gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.7,                                    
        max_output_tokens=4096,
    )

---

* 원하는 **`출력 구조 정의`**

  <br>

  * `format_parser_output`함수 = 파서 출력 → `dict` 형식으로 변환 → 출력 형식을 지정하기 위해 사용

In [ ]:
# 원하는 데이터 구조 정의

def format_parser_output(parser_output: Dict[str, Any]) -> None:
    """
    파서(parser)의 출력 결과를 사람이 읽기 좋도록 변환하여 출력합니다.

    이 함수는 Langchain 등의 도구에서 반환된 출력 결과를 처리할 때 사용됩니다.
    parser_output 딕셔너리의 각 값이 pandas DataFrame이라고 가정하고,
    이를 to_dict() 메서드를 사용해 딕셔너리로 변환한 후 보기 좋게 출력합니다.
    width=4 줄 너비를 설정, compact=True는 출력 형식을 더 간결하게 만듭니다.
    pprint는 nested 구조의 데이터를 들여쓰기와 함께 잘 정리해서 보여줍니다. 

    Args:
        parser_output (Dict[str, Any]): 파서의 출력 결과로, 
            key는 문자열이고 value는 pandas DataFrame 객체라고 가정합니다.

    Returns:
        None: 변환된 결과를 콘솔에 예쁘게 출력만 하며, 반환값은 없습니다.

    예외:
        - parser_output의 값이 DataFrame이 아닐 경우 AttributeError가 발생할 수 있습니다.
    """
    
    for key, value in parser_output.items():
        try:
            # 만약 value가 pandas DataFrame이면 to_dict() 메서드 사용
            parser_output[key] = value.to_dict()
        except AttributeError:
            # 예외 처리 및 경고 메시지 추가
            # AttributeError가 발생하면 DataFrame이 아닌 값이므로 경고 메시지를 출력
            print(f"경고: '{key}'의 값은 pandas DataFrame이 아닙니다. DataFrame이 아닌 값: {value}")
        except Exception as e:
            # 그 외의 예외 처리 및 에러 메시지 추가 
            print(f"예외 발생: {str(e)} - '{key}' 처리 중 오류가 발생했습니다.")

    # 가독성 좋은 형태로 출력
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

* `data/` > `titanic.csv` 데이터 불러오기 → `DataFrame` 로드 → `df 변수`에 할당하기

* `PandasDataFrameOutputParser` 사용 → `DataFrame` 파싱하기

In [ ]:
# 원하는 Pandas DataFrame 정의하기

df = pd.read_csv("../data/titanic.csv")
df.head()

<small>

* 셀 출력 (2.1s)

  * ![타이타닉 df.head()](../data/titanic_head.png)

In [ ]:
# 파서 설정 프롬프트 템플릿에 지시사항 넣기
parser_df = PandasDataFrameOutputParser(dataframe=df)

# 파서의 지시사항 출력해보기
print(parser_df.get_format_instructions())

<small>

* 셀 출력 (0.0s)

    ```markdown
    The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
    1. The column names are limited to the possible columns below.
    2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
    3. Remember that arrays are optional and not necessarily required.
    4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

    As an example, for the formats:
    1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
    2. String "row:1" is a well-formatted instance which gets row 1.
    3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a possible column.
    4. String "row:1[num_legs]" is a well-formatted instance which gets row 1, but for just column num_legs, where num_legs is a possible column.
    5. String "mean:num_legs[1..3]" is a well-formatted instance which takes the mean of num_legs from rows 1 to 3, where num_legs is a possible column and mean is a valid Pandas DataFrame operation.
    6. String "do_something:num_legs" is a badly-formatted instance, where do_something is not a valid Pandas DataFrame operation.
    7. String "mean:invalid_col" is a badly-formatted instance, where invalid_col is not a possible column.

    Here are the possible columns:
    
        ```
        PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked
        ```

    ```

---

* 컬럼에 대한 값을 조회하는 예시

In [ ]:
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key, value in parser_output.items():
        try:
            # 만약 value가 pandas DataFrame이면 to_dict() 메서드 사용
            parser_output[key] = value.to_dict()
        except AttributeError:
            # 예외 처리 및 경고 메시지 추가
            # AttributeError가 발생하면 DataFrame이 아닌 값이므로 경고 메시지를 출력
            print(f"경고: '{key}'의 값은 pandas DataFrame이 아닙니다. DataFrame이 아닌 값: {value}")
        except Exception as e:
            # 그 외의 예외 처리 및 에러 메시지 추가 
            print(f"예외 발생: {str(e)} - '{key}' 처리 중 오류가 발생했습니다.")

    # 가독성 좋은 형태로 출력
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

# 열 작업 예시
df_query = "Age column 을 조회해 주세요."


# 프롬프트 템플릿 설정하기
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],                                                      # 입력 변수 설정
    partial_variables={
        "format_instructions": parser_df.get_format_instructions()
    },                                                                              # 부분 변수 설정
)

# 체인 생성
chain_df = prompt | gemini_lc | parser_df

# 체인 실행
parser_output = chain_df.invoke({"query": df_query})

# 출력
format_parser_output(parser_output)


<small>

* 셀 출력 (1.1s)

    ```python
    {'Age': {0: 22.0,
            1: 38.0,
            2: 26.0,
            3: 35.0,
            4: 35.0,
            5: nan,
            6: 54.0,
            7: 2.0,
            8: 27.0,
            9: 14.0,
            10: 4.0,
            11: 58.0,
            12: 20.0,
            13: 39.0,
            14: 14.0,
            15: 55.0,
            16: 2.0,
            17: nan,
            18: 31.0,
            19: nan}}
    ```

---

* 첫번쩨 행 검색해보기

In [ ]:
# 행 조회하기
df_query = "Retrieve the first row."

# 체인 실행
parser_output = chain_df.invoke({"query": df_query})

# 결과 출력
format_parser_output(parser_output)

<small>

* 셀 출력 (0.9s)

    ```python
    {'0': {'Age': 22.0,
        'Cabin': nan,
        'Embarked': 'S',
        'Fare': 7.25,
        'Name': 'Braund, '
                'Mr. '
                'Owen '
                'Harris',
        'Parch': 0,
        'PassengerId': 1,
        'Pclass': 3,
        'Sex': 'male',
        'SibSp': 1,
        'Survived': 0,
        'Ticket': 'A/5 '
                    '21171'}}
    ```

---

* **특정 열** ~ **일부 행** 의 **평균** 검색해보기

In [ ]:
# row 0 ~ 4의 평균 나이 구하기
mean_row_0_4 = df["Age"].head().mean()

print(mean_row_0_4)                         # 31.2
print(type(mean_row_0_4))                   # <class 'numpy.float64'>

In [ ]:
# 임의의 Pandas DataFrame 작업 예시


# 행의 수 제한하기
df_query = "Retrieve the average of the Ages from row 0 to 4."
#df_query ="0~4행의 연령 평균을 검색합니다."

# 체인 실행
parser_output = chain_df.invoke({"query": df_query})

# 결과 출력
print(parser_output)                        # {'mean': np.float64(31.2)} (1.5s)

---

* 요금(Fare) 에 대한 평균 가격을 산정하는 예시

In [ ]:
# 잘못 형식화된 쿼리의 예시

df_query = "Calculate average `Fare` rate."
# df_query = "평균 `요금`을 계산합니다."

# 체인 실행
parser_output = chain_df.invoke({"query": df_query})

# 결과 출력
print(parser_output)                        # {'mean': np.float64(22.19937)} (1.0s)

<small>

* 잘못 형식화되었다고 하는 이유:

  * `"Calculate average Fare rate."`라는 쿼리에서 **`백틱(```)`** 을 사용한 것 자체는 문법적으로 문제가 될 수 있음.
  * 특히, `Langchain`과 같은 툴에서 쿼리 형식을 처리할 때 백틱을 어떻게 해석할지 정확하게 알지 못할 경우, 문제가 발생할 수 있음. 
    * 예를 들어, 일부 모델은 백틱을 특수 문자로 처리하려 할 수 있어서 쿼리의 의도와 맞지 않게 처리될 수 있음.

* 해결책_1: **백틱을 제거** 쿼리 사용

  * **`백틱(```)`** 은 컬럼 이름을 나타내는 표기법으로, 일부 환경에서는 불필요하거나 문제가 될 수 있음.
  * 쿼리에서 컬럼명을 명확히 나타내기 위해 백틱을 사용하지 않고 쿼리를 작성하는 것이 더 안전할 수 있음

    * 예시:

    ```python
    df_query = "Calculate average Fare rate."
    ```

  ---

* 해결책_2: **동적 쿼리** 생성
  * 컬럼 이름을 동적으로 처리하려면, 쿼리에서 컬럼명을 직접 입력하는 대신 동적 문자열 포매팅을 사용하여 코드를 좀 더 유연하게 만들 수 있음.
  * 이렇게 하면 타이타닉 데이터의 다른 컬럼을 쉽게 처리할 수 있음
  
  * 에시_1: 동적 퀴레 생성
  
    ```python
    column_name = "Fare"
    df_query = f"Calculate average `{column_name}` rate."
    ```

  ---

  * 예시_2: 함수를 통해 동적 쿼리 포매팅
  
  ```python
  # 컬럼명 리스트를 자동으로 생성하는 함수
  def get_column_names(df: pd.DataFrame) -> list:
      """
      데이터프레임에서 컬럼명만 추출하여 리스트로 반환합니다.
      
      Args:
          df (pd.DataFrame): pandas 데이터프레임
      
      Returns:
          list: 데이터프레임의 컬럼명 리스트
      """
      return df.columns.tolist()

  # 사용 예시
  columns = get_column_names(df)
  print(columns)  # 결과: ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
  print(type(columns))            # <class 'list'>
  ```

In [ ]:
# 컬럼명을 리스트로 자동 포매팅해보기

def get_column_names(df: pd.DataFrame) -> list:
    """
    데이터프레임에서 컬럼명만 추출하여 리스트로 반환합니다.
    
    Args:
        df (pd.DataFrame): pandas 데이터프레임
    
    Returns:
        list: 데이터프레임의 컬럼명 리스트
    """
    return df.columns.tolist()

# 사용 예시
columns = get_column_names(df)
print(columns)                  # ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
print(type(columns))            # <class 'list'>

In [ ]:
# 자동 포매팅된 함수에서 컬럼 꺼내기
column_name = "Fare"        # 원하는 컬럼명 = Fare

# 컬럼 리스트에서 해당 컬럼을 찾아서 쿼리에 넣기
for i in columns:
    if i == column_name:
        df_query = f"Calculate average `{column_name}` rate."
        column_found = True
        break
    
# 컬럼을 찾지 못한 경우 예외 처리
if not column_found:
    error_message = f"Error: '{column_name}' column not found in the DataFrame."
    print(error_message)
else:
    # 쿼리 출력
    print(df_query)                         # "Calculate average `Fare` rate."
    
    # 체인 실행 (예시로 쿼리 실행)
    try:
        parser_output = chain_df.invoke({"query": df_query})
        print(parser_output)                # {'mean': np.float64(13.0)}
    except Exception as e:
        print(f"Error during chain invocation: {str(e)}")


<small>

* **예시 출력**:

  * **예외 처리_1: 컬럼이 없을 경우** (예: `"Fare"` 컬럼이 없다면):

    ```python
      Error: 'Fare' column not found in the DataFrame.
    ```

  * **예외 처리_2: 체인 실행 중 에러가 발생한 경우**:

    ```python
      Error during chain invocation: [에러 메시지 내용]
    ```

  * **컬럼을 찾고 쿼리 실행이 성공할 경우**:

    ```python
      Calculate average `Fare` rate.
      {'mean': np.float64(13.0)} (1.0s)
    ```

* **결론**:

  * 이렇게 예외 처리 및 디버깅 메시지를 추가하면, **컬럼이 존재하지 않거나** **체인 실행 중 오류가 발생**했을 때, 쉽게 문제를 파악하고 수정 가능
  * `try-except`와 `if` 문을 활용한 예외 처리가 디버깅을 더 용이하게 만듦

In [ ]:
# 체인 실행
parser_output = chain_df.invoke({"query": df_query})

# 결과 출력
print(parser_output)                        # {'mean': np.float64(22.19937)} (1.0s)

In [ ]:
# 결과 검증
df["Fare"].mean()                          # np.float64(22.19937)

<small>

* 결과 
  * 모두 `np.float64(h22.19937)` 로 동일하게 나옴

---

<small>

* *next: DatetimeOutputParser*